In [2]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

# Load AIS data (example CSV with vessel details and positions)
ais_data = pd.read_csv('AIS_2024_01_23.csv')

# Convert BaseDateTime to datetime format for time-based sorting
ais_data['BaseDateTime'] = pd.to_datetime(ais_data['BaseDateTime'])

# Sort by MMSI and BaseDateTime to analyze each vessel's movement sequentially
ais_data = ais_data.sort_values(['MMSI', 'BaseDateTime'])

# Detect anomalies in AIS data based on speed and course changes
# Detect anomalies in AIS data based on relative speed changes and course changes
def detect_anomalies(df):
    anomalies = []
    unique_vessels = df['MMSI'].unique()

    for vessel in unique_vessels:
        vessel_data = df[df['MMSI'] == vessel].reset_index(drop=True)

        for i in range(1, len(vessel_data)):
            previous_speed = vessel_data['SOG'].iloc[i - 1]
            current_speed = vessel_data['SOG'].iloc[i]
            speed_change = abs(current_speed - previous_speed)
            course_change = abs(vessel_data['COG'].iloc[i] - vessel_data['COG'].iloc[i - 1])

            # Detect speed anomalies based on a 50% change relative to the previous speed
            if previous_speed >= 20 and speed_change > 0.45 * previous_speed and course_change > 30:
                anomalies.append(vessel_data.iloc[i])
            # Detect course anomalies
            #elif previous_speed >= 30 and course_change > 30:
                #anomalies.append(vessel_data.iloc[i])

    return pd.DataFrame(anomalies)

# Detect anomalies
anomalies_df = detect_anomalies(ais_data)
print("Anomalies detected:")
print(anomalies_df)



Anomalies detected:
          MMSI        BaseDateTime       LAT       LON   SOG    COG  Heading  \
100  200014202 2024-01-23 14:21:54  29.60747 -89.88319   0.0   43.5    511.0   
298  200014202 2024-01-23 21:19:23  29.60596 -89.87661   0.0  169.7    511.0   
87   232142000 2024-01-23 05:20:14  25.54057 -78.80176  45.3  221.4    511.0   
88   232142000 2024-01-23 05:44:06  25.80934 -80.16882   8.5  329.8    511.0   
168  232142000 2024-01-23 08:18:54  29.69324 -82.96946  80.3   19.0    511.0   
..         ...                 ...       ...       ...   ...    ...      ...   
259  982000000 2024-01-23 11:51:22  29.54384 -89.76753   2.4  267.7    511.0   
385  982000000 2024-01-23 16:19:50  29.60327 -89.84782   5.4   61.0    511.0   
401  982000000 2024-01-23 16:37:39  29.60552 -89.85261   1.7  127.3    511.0   
439  982000000 2024-01-23 17:50:26  29.60512 -89.85257   1.8  299.9    511.0   
595  982000000 2024-01-23 22:38:19  29.54421 -89.76861   0.7  249.8    511.0   

              Vesse

In [3]:
# Install necessary packages
!pip install geemap
!pip install earthengine-api
# Install Earth Engine and Geemap
!pip install earthengine-api geemap


# Authenticate and initialize Earth Engine
import ee
import geemap

# Trigger the authentication
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize(project='ee-oilspilldetectionharshasri')


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
def plot_histogram(image, region, title):
    try:
        array = image.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=region,
            scale=30,
            maxPixels=1e9
        ).get('VV')

        # Check if array is empty
        if array is None:
            print("No VV data available for histogram plot.")
            return

        array = np.array(array.getInfo())
        plt.hist(array, bins=50, color='blue', alpha=0.7)
        plt.title(title)
        plt.xlabel('VV (dB)')
        plt.ylabel('Frequency')
        plt.grid()
        plt.show()
    except Exception as e:
        print(f"Error plotting histogram: {e}")

In [5]:
from datetime import datetime

# Function to parse date from BaseDateTime
def extract_date(base_date_time):
    # Convert to string if it’s a Timestamp
    if isinstance(base_date_time, pd.Timestamp):
        base_date_time = base_date_time.strftime('%Y-%m-%dT%H:%M:%S')
    else:
        base_date_time = str(base_date_time)

    # Parse the date
    date_time_obj = datetime.strptime(base_date_time, '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime('%Y-%m-%d')


In [6]:
import tensorflow as tf

# Define jaccard_coef if not already defined
def jaccard_coef(y_true, y_pred):
    y_true_flatten = tf.reshape(y_true, [-1])
    y_pred_flatten = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_flatten * y_pred_flatten)
    sum_ = tf.reduce_sum(y_true_flatten) + tf.reduce_sum(y_pred_flatten)
    smooth = 1e-6  # To avoid division by zero
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac


In [7]:
from tensorflow.keras.models import load_model
import os

# Ensure segmentation_models uses TensorFlow backend
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

# Load the models
hybrid_model = load_model(r"C:\Users\harsh\OneDrive\Desktop\Capstone\hybrid_model.h5")
unet_model = load_model(r"C:\Users\harsh\OneDrive\Desktop\Capstone\U_net.h5")
deeplab_model = load_model(
    r"C:\Users\harsh\OneDrive\Desktop\Capstone\deeplab_model.h5",
    custom_objects={
        "jaccard_coef": jaccard_coef,  # Ensure this function is defined in your script
        "DiceLoss": sm.losses.DiceLoss(),  # Initialize the DiceLoss object
        "CategoricalFocalLoss": sm.losses.CategoricalFocalLoss()  # Initialize the CategoricalFocalLoss object
    }
)


Segmentation Models: using `tf.keras` framework.


In [8]:
def preprocess_image_for_model(image_np):
    """ Preprocess the NumPy array image to match model input requirements """
    # Remove any extra dimensions if present (e.g., shape (256, 256, 1, 6) -> (256, 256, 2))
    if image_np.ndim == 4:
        image_np = image_np[:, :, :, 0]

    # Resize to 256x256
    resized_image = cv2.resize(image_np, (256, 256))

    # Ensure the image has three channels (duplicate one if necessary)
    if resized_image.shape[-1] == 2:
        resized_image = np.repeat(resized_image[:, :, :1], 3, axis=-1)
    elif resized_image.shape[-1] == 1:
        resized_image = np.repeat(resized_image, 3, axis=-1)

    # Add batch dimension
    return np.expand_dims(resized_image, axis=0)

In [9]:
def confirm_oil_spill(hybrid_model, unet_model, deeplab_model, image):
    """ 
    Confirm oil spill detection using Hybrid, UNet, and DeepLabV3+ models. 
    Requires agreement among all models.
    """
    processed_image = preprocess_image_for_model(image)

    #processed_image = preprocess_image_for_model(image)
        
        # Get predictions from both models
    unet_prediction = unet_model.predict(processed_image)
    deeplab_prediction = deeplab_model.predict(processed_image)
    hybrid_prediction = hybrid_model.predict(processed_image)

    # Assuming '1' corresponds to oil spill for both models
    unet_result = np.argmax(unet_prediction) == 1
    deeplab_result = np.argmax(deeplab_prediction) == 1
    hybrid_result = np.argmax(hybrid_prediction) == 1

    # Confirm only if both models agree
    #return unet_result 

    # Confirm only if all models agree
    return hybrid_result and unet_result and deeplab_result


In [10]:
from tensorflow.keras.models import load_model
import geemap
import ee
import numpy as np
import cv2
from IPython.display import display

def process_anomaly(row):
    lat = row['LAT']
    lon = row['LON']
    base_date_time = row['BaseDateTime']
    date = extract_date(base_date_time)
    anomaly_date = ee.Date(date)
    print(f"Processing anomaly at {lat}, {lon} on {date}")

    roi = ee.Geometry.Point(lon, lat).buffer(10000).bounds()
    detected_spills = []

    # Sentinel-1 collection
    sen1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(anomaly_date, anomaly_date.advance(1, 'day')) \
        .filterBounds(roi) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # for x in sen1.size():
    #     print(x)

    if sen1.size().getInfo() == 0:
        print("No Sentinel-1 VV or VH images available. Trying Sentinel-2.")
        sen1 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterDate(anomaly_date, anomaly_date.advance(1, 'day')) \
            .filterBounds(roi) \
            .filter(ee.Filter.eq('CLOUDY_PIXEL_PERCENTAGE', 0))

        if sen1.size().getInfo() == 0:
            print("No Sentinel-2 images available for this date and location.")
            return None

        sen1_image = sen1.select(['B4', 'B8']).mosaic()
    else:
        sen1_image = sen1.select(['VV', 'VH']).mosaic()
    

    despeckled = sen1_image.focal_mean(100, 'square', 'meters')
    vv_dark = despeckled.select('VV').lt(-22)
    vh_dark = despeckled.select('VH').lt(-18)
    oil_spill_detection = vv_dark.And(vh_dark)

    mask = oil_spill_detection.updateMask(oil_spill_detection)
    area = mask.multiply(ee.Image.pixelArea().divide(1e6))
    oil_spill_area = ee.Number(
        area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=roi,
            scale=100
        ).values().get(0)
    ).getInfo()

    if oil_spill_area > 0:
        print(f"Detected Oil Spill Area (sq. km): {oil_spill_area}")

        despeckled_np = geemap.ee_to_numpy(despeckled, bands=['VV', 'VH'], region=roi, scale=100)

        # Ensure image preprocessing for models
        #processed_image = preprocess_image_for_model(despeckled_np)

        # Confirm using all three models
        if confirm_oil_spill(hybrid_model, unet_model, deeplab_model, despeckled_np):
            detected_spills.append(oil_spill_area)

            # oil_spill_vis_params = {
            #     'min': 0,
            #     'max': 1,
            #     'palette': ['blue', 'cyan', 'yellow', 'red']
            # }

            # Visualization
            Map = geemap.Map()
            Map.centerObject(roi)
            Map.addLayer(sen1_image, {'bands': ['VV', 'VH'], 'min': -30, 'max': 0}, f'Sentinel-1 Image {anomaly_date.getInfo()}')
            Map.addLayer(despeckled.clip(roi), {}, 'Despeckled Image', False)
            Map.addLayer(oil_spill_detection.clip(roi), {}, 'Oil Spill Detection', False)
            oil_spill_vector = mask.reduceToVectors(
                geometry=roi,
                scale=100
            )
            Map.addLayer(oil_spill_vector, {}, 'Oil Spill Vector', False)

    if detected_spills:
        print(f"Oil spills detected over the days: {detected_spills}")
        Map.addLayerControl()
        return Map
    else:
        print("No significant oil spills detected across the checked dates.")
        return None


In [11]:
# Iterate over each anomaly in the DataFrame and process
for index, row in anomalies_df.iterrows():
    result_map = process_anomaly(row)
    if result_map:
        display(result_map)  # This displays the map in interactive environments

Processing anomaly at 29.60747, -89.88319 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 29.60596, -89.87661 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 25.54057, -78.80176 on 2024-01-23
No significant oil spills detected across the checked dates.
Processing anomaly at 25.80934, -80.16882 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 29.69324, -82.96946 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 25.76805, -80.13911 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processi

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Processing anomaly at 49.14809, -123.83936 on 2024-01-23
Detected Oil Spill Area (sq. km): 102.47828815107994
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Oil spills detected over the days: [102.47828815107994]


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Processing anomaly at 42.38143, -79.81747 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 42.54411, -79.62458 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 42.80293, -79.32444 on 2024-01-23
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 49.19839, -123.10066 on 2024-01-23
Detected Oil Spill Area (sq. km): 14.989150819751407
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
No significant oil spills detected across the checked dates.
Processing anomaly at 49.19758, -122.92495 on 2024-01-23
Detected Oil Spill Area (sq. km): 0.6893922550532315
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1